In [8]:
import json
import time
import os
import sklearn_crfsuite

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

nertagger = NerTagger()
word_level_ner = WordLevelNerTagger()

In [12]:
# Reading in files from the distributed corpus:

files = {}

with open('divided_corpus.txt', 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [17]:
startTime = time.perf_counter()
for file in files:
    if int(files[file]) in (1, 2, 3):
        with open("vallakohtufailid_json_flat/" + file, 'r', encoding='UTF-8') as f:
            text_import = json_to_text(f.read())
            text = Text(text_import.text)
            text.tag_layer(['sentences', 'morph_analysis'])
            nertagger.tag(text)
            word_level_ner.tag(text)
            
            text.add_layer(flatten(text['ner'], 'flat_ner'))
            text.add_layer(flatten(text['wordner'], 'flat_wordner'))
            
            removed_layers = ['sentences', 'morph_analysis', 'wordner', 'compound_tokens', 'ner', 'words', 'tokens']
            for x in removed_layers:
                text.pop_layer(x)
            
            text_to_json(text, file=os.getcwd() + "/vallakohtufailid_nertagger/" + file)
    print("Fail märgendatud.")
endTime = time.perf_counter()
print(f"1. alajaotus märgendati {endTime - startTime:0.4f} sekundiga.")

Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märge

Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märgendatud.
Fail märge